In [1]:
import pandas as pd
from haversine import haversine
import numpy as np
import datetime
import os
import json
os.getcwd()

'/home/jian/Projects/Research_zip_request/PlanetFitness_20190104'

In [2]:
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json"))
input_address=pd.read_excel("/home/jian/Projects/Research_zip_request/PlanetFitness_20190104/Planet Fitness Locations.xlsx",dtype=str)

input_address.head(2)

,list_name,store_name,address,city,state,zip_code,DMA,phone_number,latitude,longitude,Census Region
0,Planet Fitness,"Belchertown, MA",40 Turkey Hill Road,Belchertown,MA,01007,SPRINGFIELD-HOLYOKE,(413) 323-1003,42.2654754,-72.4209138,Northeast
1,Planet Fitness,"Chicopee, MA",1183 Memorial Dr,Chicopee,MA,01020,SPRINGFIELD-HOLYOKE,(413) 593-5566,42.1897195,-72.5761129,Northeast


In [3]:
input_address['zip_list_in_5_miles']=np.nan
address_zips_long=pd.DataFrame()
for i in range(len(input_address)):
    store_zip_list=[input_address['zip_code'][i].zfill(5)]
    store_lat=float(input_address['latitude'][i])
    store_lng=float(input_address['longitude'][i])
    store_center=[store_lat,store_lng]
    df_long_0=input_address.iloc[i,:].to_frame().T
    df_long_0['zip_cd_5_miles']=input_address['zip_code'][i]
    df_long_0['distance']="Store_Zip"
    
    
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=5:
            store_zip_list=store_zip_list+[zip_cd]
            df_long=input_address.iloc[i,:].to_frame().T
            df_long['zip_cd_5_miles']=zip_cd
            df_long['distance']=dist
            df_long_0=df_long_0.append(df_long)
            
            
    address_zips_long=address_zips_long.append(df_long_0,ignore_index=True)
    
    store_zip_list=list(set(store_zip_list))
    input_address['zip_list_in_5_miles'][i]=store_zip_list
    
    
    

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
writer=pd.ExcelWriter("/home/jian/Projects/Research_zip_request/PlanetFitness_20190104/Planet Fitness Locations with Zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
input_address.to_excel(writer,"address_with_zip_list",index=False)
address_zips_long.to_excel(writer,"zips_in_5_miles",index=False)
writer.save()


In [5]:
address_zips_long.shape

(20183, 14)

In [7]:
input_address['zip_code'].apply(lambda x: len(x)).unique()

array([5])

In [8]:
input_address.shape

(1578, 12)